In [1]:
import pandas as pd
from datetime import datetime

# Laad de dataset
data = pd.read_csv('./healthcare_data.csv')

# Functie om anomalieën te detecteren
def detect_anomalies(df):
    anomalies = []

    # Controleer op afwijkende scanmethoden
    for i, row in df.iterrows():
        if row['scan_method'] != 'eID' and row['timestamp'].weekday() in [5, 6]:
            anomalies.append(f"Anomalie in scanmethode voor patiënt {row['patient_id']} op {row['timestamp']}")

    # Controleer op zorg door meerdere verpleegkundigen op dezelfde dag
    for patient_id, group in df.groupby('patient_id'):
        if group['timestamp'].dt.date.nunique() > 1:
            anomalies.append(f"Meerdere verpleegkundigen zorgden voor patiënt {patient_id} op dezelfde dag")

    # Controleer op verdachte uren
    for i, row in df.iterrows():
        hour = row['timestamp'].hour
        if hour < 6 or hour > 22:
            anomalies.append(f"Verdachte uur voor bezoek aan patiënt {row['patient_id']} door verpleegkundige {row['nurse_id']}")

    # Controleer op te korte tussenpozen tussen bezoeken
    for patient_id, group in df.groupby('patient_id'):
        times = group['timestamp'].sort_values()
        for i in range(1, len(times)):
            if (times.iloc[i] - times.iloc[i-1]).total_seconds() < 2.5 * 3600:
                anomalies.append(f"Te korte tussenpozen tussen bezoeken aan patiënt {patient_id}")

    return anomalies

# Converteer timestamps naar datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Detecteer anomalieën
anomalies = detect_anomalies(data)

# Print de resultaten
for anomaly in anomalies:
    print(anomaly)


Meerdere verpleegkundigen zorgden voor patiënt 12345 op dezelfde dag
Te korte tussenpozen tussen bezoeken aan patiënt 12345
Te korte tussenpozen tussen bezoeken aan patiënt 12345
